In [9]:
import numpy as np
import altair as alt
import pandas as pd
import os

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

## Load Data

In [17]:
sales_df = pd.read_csv("../data/raw/sales_and_customer_insights.csv", parse_dates=True)

## Basic Data Overview

In [18]:
sales_df.head()

,Customer_ID,Product_ID,Transaction_ID,Purchase_Frequency,Average_Order_Value,Most_Frequent_Category,Time_Between_Purchases,Region,Churn_Probability,Lifetime_Value,Launch_Date,Peak_Sales_Date,Season,Preferred_Purchase_Times,Retention_Strategy
0,CUST_9HOS83,PROD_IK97D1,TRANS_II1DZG,17,172.57,Electronics,45,South America,0.98,952.81,2020-03-14,2023-09-11,Winter,Afternoon,Loyalty Program
1,CUST_AJU17N,PROD_UNN7KP,TRANS_9HJF7I,10,64.89,Clothing,6,South America,0.66,5427.51,2022-10-15,2023-01-02,Spring,Afternoon,Discount
2,CUST_11XNYF,PROD_0XEW2W,TRANS_OT96OM,3,120.38,Sports,23,Asia,0.60,3994.80,2021-11-30,2023-04-06,Winter,Evening,Loyalty Program
3,CUST_IGH8G3,PROD_3IIAJN,TRANS_45V00G,12,70.34,Clothing,5,North America,0.78,4880.01,2022-03-20,2023-03-23,Spring,Evening,Discount
4,CUST_OK6PUM,PROD_VMIWD2,TRANS_ZAK760,18,42.39,Electronics,10,North America,0.52,8557.70,2022-11-09,2023-10-28,Spring,Morning,Discount


In [19]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               10000 non-null  object 
 1   Product_ID                10000 non-null  object 
 2   Transaction_ID            10000 non-null  object 
 3   Purchase_Frequency        10000 non-null  int64  
 4   Average_Order_Value       10000 non-null  float64
 5   Most_Frequent_Category    10000 non-null  object 
 6   Time_Between_Purchases    10000 non-null  int64  
 7   Region                    10000 non-null  object 
 8   Churn_Probability         10000 non-null  float64
 9   Lifetime_Value            10000 non-null  float64
 10  Launch_Date               10000 non-null  object 
 11  Peak_Sales_Date           10000 non-null  object 
 12  Season                    10000 non-null  object 
 13  Preferred_Purchase_Times  10000 non-null  object 
 14  Retenti

In [7]:
sales_df.describe()

,Purchase_Frequency,Average_Order_Value,Time_Between_Purchases,Churn_Probability,Lifetime_Value
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,9.955700,110.006022,46.885300,0.501552,5031.930567
std,5.526209,52.096870,24.711746,0.288289,2865.420299
min,1.000000,20.010000,5.000000,0.000000,100.160000
25%,5.000000,65.262500,25.000000,0.250000,2518.775000
50%,10.000000,109.930000,47.000000,0.500000,5012.180000
75%,15.000000,155.480000,68.000000,0.750000,7526.232500
max,19.000000,199.960000,89.000000,1.000000,9999.760000


## Charts For Dashboard Markup

In [13]:
sales_100_sample_df = sales_df.sample(100, random_state = 448)
sales_1000_sample_df = sales_df.sample(1000, random_state = 448)

In [16]:
story_2_example = alt.Chart(sales_100_sample_df).mark_circle(size = 100).encode(
    alt.X('Lifetime_Value', title='Lifetime Value'),
    alt.Y('Churn_Probability', title='Churn Probability'),
    color = 'Retention_Strategy',
).properties(
    title = 'Retension Strategies (100 point sample)',
    height = 400,
    width = 400,
)
story_2_example

alt.Chart(...)

In [22]:
story_3_example = alt.Chart(sales_1000_sample_df).mark_rect(color = "viridis").encode(
    alt.X('Season', title='Season'),
    alt.Y('Most_Frequent_Category', title='Product Type'),
    color = "Purchase_Frequency:Q"
).properties(
    title = 'Retension Strategies (1000 point sample)',
    height = 400,
    width = 400,
)
story_3_example

alt.Chart(...)